# Enterprise Firewall Statistics APIs

<p><font size="4"><div class="lev1 toc-item"><a href="#Objective" data-toc-modified-id="Objective-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Objective</a></div><div class="lev1 toc-item"><a href="#Resource-URL-Structure-Components" data-toc-modified-id="Resource-URL-Structure-Components-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Resource URL Structure Components</a></div><div class="lev1 toc-item"><a href="#API-Authentication" data-toc-modified-id="API-Authentication-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>API Authentication</a></div><div class="lev1 toc-item"><a href="#Query-Payload" data-toc-modified-id="Query-Payload-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Query Payload</a></div><div class="lev1 toc-item"><a href="#Aggregation-API-Request" data-toc-modified-id="Aggregation-API-Request-2"><span class="toc-item-num">5&nbsp;&nbsp;</span>Aggregation API Request</a></div><div class="lev1 toc-item"><a href="#InfluxDB-and-Grafana" data-toc-modified-id="InfluxDB-and-Grafana-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>InfluxDB and Grafana</a></div></font>

# Objective


<font size="4"> 
- How to retrieve the Enterprise Firewall Inspect session counter values
<br><br>
- Store firewall inspect count values in a Time Series Database (TSDB) **Influx DB**
<br><br>
- Plot statistics on to **Grafana** Dashboard 
</font>

# Resource URL Structure Components

<font size="4">
    
- In order to retrieve firewall statistics either <b>Inspect count</b> or <b>Drop count</b> we need to use the resource URL `/statistics/fwall/aggregation`
<br><br>
- <b>API end point:</b> `https://vmanage-ip:port/dataservice/statistics/fwall/aggregation` 

</font><br>
    
![resource-url](images/resource_url.png)

<br>
<font size="4">Now let’s start using the python script to fetch the Enterprise Firewall Inspect sessions count by using below steps
<br><br>
    
- Login and Authenticate with vManage instance
<br><br>
- Build query to specify the rules on how to collect Enterprise Firewall statistics.
<br><br>
- Perform the **POST** operation by sending query in payload
</font>


# API Authentication

<font size="4"> 
Let's start by going through Authentication Class which is used to get authenticated with vManage and receive cookie, token which are used for authentication in subsequent API calls.
<br><br>
**API URL:** `https://vmanage-ip:port/j_security_check`
    
Input parameters for Authentication Class:
<br><br>
 - vManage FQDN/IP Address
 - vManage Port Number
 - Username
 - Password

<br>
On successful POST operation we get <b>JSESSIONID/Cookie</b> which is used in subsequent API calls by client. By nature REST API is stateless so we use JSESSIONID for doing session tracking.
</font>

In [ ]:
from influxdb import InfluxDBClient
import requests
import json
import time

from requests.packages.urllib3.exceptions import InsecureRequestWarning

vmanage_host = "198.18.1.10"
vmanage_port = "443"
vmanage_username = "admin"
vmanage_password = "admin"

requests.packages.urllib3.disable_warnings()

class Authentication:

    @staticmethod
    def get_jsessionid(vmanage_host, vmanage_port, username, password):
        api_url = "/j_security_check"
        base_url = "https://%s:%s"%(vmanage_host, vmanage_port)
        url = base_url + api_url
        payload = {'j_username' : username, 'j_password' : password}
        
        response = requests.post(url=url, data=payload, verify=False)
        try:
            cookies = response.headers["Set-Cookie"]
            jsessionid = cookies.split(";")
            print("\nAuthentication is successful")
            return(jsessionid[0])
        except:
            print("\nAuthentication failed, No valid JSESSION ID returned")
            exit()
       
    @staticmethod
    def get_token(vmanage_host, vmanage_port, jsessionid):
        headers = {'Cookie': jsessionid}
        base_url = "https://%s:%s"%(vmanage_host, vmanage_port)
        api_url = "/dataservice/client/token"
        url = base_url + api_url     
        response = requests.get(url=url, headers=headers, verify=False)
        if response.status_code == 200:
            return(response.text)
        else:
            return None


try:
    Auth = Authentication()
    jsessionid = Auth.get_jsessionid(vmanage_host,vmanage_port,vmanage_username,vmanage_password)
    print("\nSession Cookie: ", jsessionid)
    token = Auth.get_token(vmanage_host,vmanage_port,jsessionid)
    print("\nSession Token: ", token)
except Exception as e:
    print(e)

# Query Payload

<font size="4"> 
- In order to retrieve the firewall statistics we need to send Query which describes how statistics should be aggregated. 
<br><br>
- Now let’s define the Query which can be used to collect firewall statistics using POST operation on Resource URL `/dataservice/statistics/fwall/aggregation`
<br><br>
    
<b>Example-1</b>

- Below example query retrieves Enterprise Firewall Inspect count values for last 24 hours from all network devices in fabric. 
<br><br>
- The aggregation portion of the query determines how data is bucketized/grouped.
<br><br>
- In this example, the Enterprise Firewall inspect counter statistics on each SD-WAN router are aggregated in 30 mins intervals across the span of 24 hours.
<br><br>
    
<pre>
{
  "aggregation": {                                   # Defines how statistics are bucketized
    "metrics": [
      {
        "property": <b>"fw_total_insp_count"</b>,  
        "type": "sum"                                
      }
    ],
    "field": [
      {
        "property": <b>"vdevice_name"</b>,
        "dataType": "string"
      }
    ],
    "histogram": {                                   # Time interval is 30 mins
      "property": "entry_time",
      "type": "minute",
      "interval": <b>30</b>,
      "order": "asc"
    }
  },
  "query": {
    "condition": "AND",
    "rules": [
      {
        "value": [
          <b>"24"</b>                                # Statistics from last 24 hours
        ],
        "field": "entry_time",
        "type": "date",
        "operator": "last_n_hours"
      }
    ]
  }
}

</pre>

<b>Example-2</b>

- Below query retrieves the Enterprise Firewall Drop count for last 24 hours from all network devices in fabric and statistics are aggregated in 30 mins intervals across the span of 24 hours.

<pre>
{
  "aggregation": {
    "metrics": [
      {
        "property": <b>"fw_total_drop_count"</b>,
        "type": "sum"
      }
    ],
    "histogram": {
      "property": "entry_time",
      "type": "minute",
      "interval": 30,
      "order": "asc"
    }
  },
  "query": {
    "condition": "AND",
    "rules": [
      {
        "value": [
          "24"
        ],
        "field": "entry_time",
        "type": "date",
        "operator": "last_n_hours"
      }
    ]
  }
}
</pre>

<b>Example-3</b>

- Below query retrieves the Enterprise Firewall Inspect count for last 1 hour and aggregated for 1 minute intervals.

<pre>
{
  "aggregation": {
    "metrics": [
      {
        "property": "fw_total_insp_count",
        "type": "sum"
      }
    ],
    "histogram": {
      "property": "entry_time",
      "type": "minute",
      <b>"interval": 1</b>,
      "order": "asc"
    }
  },
  "query": {
    "condition": "AND",
    "rules": [
      {
        "value": [
          "1"
        ],
        "field": "entry_time",
        "type": "date",
        "operator": "last_n_hours"
      }
    ]
  }
}
</pre>
</font> 

# Aggregation API Request

<font size="4">   
- Using the **Example-1** query let's perform POST request to retrieve the Enterprise Firewall Inspect count statistics for last 24 hours in intervals of 30 mins.
<br><br>
- Now let's have a look at the response to POST operation.
</font>

<br>

<font size="4">
    
<b>Sample Response</b>

- `entry_time` values are in epoch format and corresponding key `fw_total_insp_count` contains the Enterprise Firewall Inspect count value at that `entry_time`. 

<pre>

Firewall Inspect count

  "data": [
    {
      <b>"entry_time": 1552807800000</b>,
      "count": 92,
      <b>"fw_total_insp_count": 278</b>
    },
    {
      "entry_time": 1552806000000,
      "count": 360,
      "fw_total_insp_count": 1050
    },
    {
      "entry_time": 1552804200000,
      "count": 360,
      "fw_total_insp_count": 1052
    },
    {
      "entry_time": 1552802400000,
      "count": 360,
      "fw_total_insp_count": 1051
    },
    
</pre>
</font>

In [ ]:
payload = {
            "aggregation": {
            "metrics": [
                            {
                                "property": "fw_total_insp_count",
                                "type": "sum"            
                            }
                       ],
            "field": [
                          {
                                "property": "vdevice_name",
                                "dataType": "string"
                          }
                     ],
            "histogram": 
                          {
                                "property": "entry_time",
                                "type": "minute",
                                "interval": 30,
                                "order": "asc"
                          }
                    },
            "query": 
                    {
                        "condition": "AND",
                        "rules": [
                                    {
                                        "value": ["24"],
                                        "field": "entry_time",
                                        "type": "date",
                                        "operator": "last_n_hours"
                                    }
                                ]
                    }
         }

if token is not None:
    headers = {'Content-Type': "application/json",'Cookie': jsessionid, 'X-XSRF-TOKEN': token}
else:
    headers = {'Content-Type': "application/json",'Cookie': jsessionid}

base_url = "https://%s:%s"%(vmanage_host, vmanage_port)
api_url = "/dataservice/statistics/fwall/aggregation"

url = base_url + api_url

response = requests.post(url, headers=headers, data=json.dumps(payload), verify=False)

if response.status_code == 200:

    items = response.json()['data']

    print("\nFirewall inspect session count first sample\n",items[0])

else:
    print("\nFetching firewall inspect session count failed\n")
    print(response.text)

# InfluxDB and Grafana

<font size="4"> 
- InfluxDB is a time series database which is optimized for fast, high-availability storage and retrieval of time series data in fields such as operations monitoring, real-time analytics.
<br><br>
- InfluxDB provides a SQL-like language with built-in time functions for querying a data structure composed of measurements, series, and points
<br><br>
- Grafana is used to build monitoring and analytics dashboards.
<br><br>
- Here, we are using InfluxDB as a data source for Grafana to plot the statistics on dashboard.
</font>


In [ ]:
#login credentials for InfluxDB

USER = 'admin'
PASSWORD = 'admin'
DBNAME = 'firewall_inspect'

#InfluxDB host details

host='localhost'
port=8111

series = []
total_records = 0

json_body = {}

#loop over the vManage API response variable items and create records to be stored in InfluxDB

for i in items:
    json_body = { "measurement": "firewall_inspect_count",
                  "tags": {
                            "host": i["vdevice_name"],
                          },
                 "time": time.strftime('%m/%d/%Y %H:%M:%S',  time.gmtime(i['entry_time']/1000.)),
                 "fields": {
                            "value": float(i['fw_total_insp_count'])
                }
                }
    series.append(json_body)
    total_records = total_records+1

client = InfluxDBClient(host, port, USER, PASSWORD, DBNAME)

retention_policy = 'firewall_stats_retention_policy'
client.create_retention_policy(retention_policy, '10d', 3, default=True)

print("Stored Enterprise Firewall Inspect counter samples: {0}".format(total_records))
client.write_points(series, retention_policy=retention_policy)